In [9]:
import pandas as pd
import numpy as np
import datetime as dt
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc

from pathlib import Path

In [10]:
DATOS_BRUTOS = Path("../datos/brutos")

In [2]:
bpl.output_notebook()

Loading BokehJS ...

Para ejecutar estos análisis primero se tiene que ejecutar el notebook del archivo "datos_historicos.ipynb", ese notebook permite descargar los archivos de datos con los que trabaja este notebook.

In [15]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "05/25/2020"

In [16]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [17]:
fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [18]:
frames = []
for fecha in fechas_csv:
    data = pd.read_csv(DATOS_BRUTOS / fecha, encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    frames.append(data)

In [19]:
len(frames)

44

In [20]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [21]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [22]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [23]:
confirmados = all_data[all_data["RESULTADO"]==1]
difuntos = confirmados[confirmados["FECHA_DEF"].notna()]

In [24]:
por_fecha = confirmados.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [25]:
fechas_validas = por_fecha.columns

In [26]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [27]:
conf_acum = por_fecha.sum().to_frame("Confirmados").reset_index()

In [28]:
conf_acum["diferencia"] = conf_acum["Confirmados"].diff()

In [29]:
conf_acum[:3]

,FECHA_ARCHIVO,Confirmados,diferencia
0,2020-04-12,4661.0,NaN
1,2020-04-13,5014.0,353.0
2,2020-04-14,5399.0,385.0


In [30]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [31]:
p.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=1,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='1036', ...)

In [32]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i])

In [33]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [34]:
bpl.show(p)

In [35]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [36]:
pday.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=1,top=conf_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='2496', ...)

In [37]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        pday.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],line_alpha=0.3)

In [38]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Casos nuevos reportados'
pday.legend.location = "top_left"

In [39]:
bpl.show(pday)

In [40]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,
               x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [41]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [42]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia en casos'
s.legend.location = "top_left"

In [43]:
bpl.show(s)

In [44]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [45]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())*100/(por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [46]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia porcentual'
s.legend.location = "top_right"

In [47]:
bpl.show(s)

In [48]:
df_fecha = []
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    conf_sint.name = fecha
    df_fecha.append(conf_sint)

In [49]:
valores = pd.concat(df_fecha,axis=1).T[fechas_validas[:-1]]

In [50]:
cambios = valores.pct_change()*100

In [51]:
fechas_casos = cambios.columns

In [52]:
coloresc = list(bpt.inferno(len(fechas_casos)))
coloresc.reverse()

In [53]:
s1 = bpl.figure(plot_width=1000,plot_height=500)

In [54]:
for i,fecha in enumerate(fechas_casos):
    t_data = cambios[fecha].to_frame("Cambio")
    t_data["Valores"] = valores[fecha]
    tf_data = t_data[t_data["Valores"].notna()].reset_index()
    tf_data["dias"] = (tf_data["index"]-fecha).dt.days
    if (np.mod(i,10)==0 or i == len(fechas_casos)-1):
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i],legend_label = fecha.strftime("%d/%m/%Y"))
    else:
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i])

In [55]:
s1.xaxis.axis_label = 'Días'
s1.yaxis.axis_label = 'Cambio porcentual diario'
s1.legend.location = "top_right"

In [56]:
bpl.show(s1)

In [57]:
por_fecha_dif = difuntos.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [58]:
fechas_validas = por_fecha_dif.columns

In [59]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [60]:
dif_acum = por_fecha_dif.sum().to_frame("Decesos").reset_index()

In [61]:
dif_acum["diferencia"] = dif_acum["Decesos"].diff()

In [62]:
dif_acum[:3]

,FECHA_ARCHIVO,Decesos,diferencia
0,2020-04-12,296.0,NaN
1,2020-04-13,332.0,36.0
2,2020-04-14,406.0,74.0


In [63]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [64]:
p.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=1,top=dif_acum["Decesos"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos reportados en las conferencias")

GlyphRenderer(id='9084', ...)

In [65]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Defunciones por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i])

In [66]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Decesos confirmados'
p.legend.location = "top_left"

In [67]:
bpl.show(p)

In [68]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [69]:
pday.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=1,top=dif_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='11069', ...)

In [70]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Decesos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],line_alpha=0.3)

In [71]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Decesos nuevos reportados'
pday.legend.location = "top_left"

In [72]:
bpl.show(pday)

In [73]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,
               x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [74]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    porcentajes = (dif_sint - por_fecha_dif.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [75]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia en casos'
s.legend.location = "top_left"

In [76]:
bpl.show(s)

In [77]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [78]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    porcentajes = (dif_sint - por_fecha_dif.sum())*100/(por_fecha_dif.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [79]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia porcentual'
s.legend.location = "top_right"

In [80]:
bpl.show(s)

In [81]:
df_fecha = []
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    dif_sint.name = fecha
    df_fecha.append(dif_sint)

In [82]:
valores = pd.concat(df_fecha,axis=1).T[fechas_validas[:-4]]

In [83]:
cambios = valores.pct_change()*100

In [84]:
fechas_casos = cambios.columns

In [85]:
coloresc = list(bpt.inferno(len(fechas_casos)))
coloresc.reverse()

In [86]:
s1 = bpl.figure(plot_width=1000,plot_height=500)

In [87]:
for i,fecha in enumerate(fechas_casos):
    t_data = cambios[fecha].to_frame("Cambio")
    t_data["Valores"] = valores[fecha]
    tf_data = t_data[t_data["Valores"].notna()].reset_index()
    tf_data["dias"] = (tf_data["index"]-fecha).dt.days
    if (np.mod(i,10)==0 or i == len(fechas_casos)-1):
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i],legend_label = fecha.strftime("%d/%m/%Y"))
    else:
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i])

In [88]:
s1.xaxis.axis_label = 'Días'
s1.yaxis.axis_label = 'Cambio porcentual diario'
s1.legend.location = "top_right"

In [89]:
bpl.show(s1)

In [90]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [91]:
pacientes = all_data.groupby(["ID_REGISTRO","ENTIDAD_UM","ENTIDAD_RES","MUNICIPIO_RES","EDAD","EMBARAZO","HABLA_LENGUA_INDIG","DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION","OTRA_COM","CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO","MIGRANTE"]).size().reset_index().drop(columns=[0])

In [92]:
varios = pacientes["ID_REGISTRO"].value_counts()[pacientes["ID_REGISTRO"].value_counts()>1].index

In [93]:
varios

Index(['1958e7', '1ad493', '17dd02', '066195', '1945dd', '1df826', '0a2098',
       '0732bc', '15d77f', '10e249',
       ...
       '185e65', '060f6d', '087d3a', '0c0cb4', '127dd4', '05e4fb', '05159a',
       '1766f4', '0c4d40', '13822c'],
      dtype='object', length=1041)

In [94]:
conf_validos = pd.merge(pacientes,por_fecha.reset_index()[~por_fecha.reset_index()["ID_REGISTRO"].isin(varios)],how="right",on="ID_REGISTRO")

In [95]:
conf_filtrados = conf_validos
#conf_filtrados = conf_validos[(conf_validos["ENTIDAD_RES"]==30)]
#conf_filtrados = conf_validos[(conf_validos["ENTIDAD_RES"]==2)&(conf_validos["MUNICIPIO_RES"]==4)]

In [96]:
conf_acum = conf_filtrados[fechas_validas].sum().to_frame("Confirmados").reset_index()

In [97]:
conf_acum["diferencia"] = conf_acum["Confirmados"].diff()

In [98]:
conf_acum[:3]

,index,Confirmados,diferencia
0,2020-04-12,4617.0,NaN
1,2020-04-13,4965.0,348.0
2,2020-04-14,5346.0,381.0


In [99]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [100]:
p.vbar(x=conf_acum["index"],bottom=1,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='19606', ...)

In [101]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [102]:
bpl.show(p)

In [103]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [104]:
s.vbar(x=conf_acum["index"],bottom=1,top=conf_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='20856', ...)

In [105]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Casos nuevos confirmados'
s.legend.location = "top_left"

In [106]:
bpl.show(s)